##### Adjusting sell side ratings to align with llm recommendations

In [ ]:
analyst_ratings = pd.read_csv("../data/sp1500_sell_side_recommendations_cleaned.csv", dtype={"cik": str})
analyst_ratings["date"] = pd.to_datetime(analyst_ratings["date"]).dt.to_period('M')

In [ ]:
min_month = analyst_ratings['date'].min()
max_month = analyst_ratings['date'].max()
min_month, max_month

In [ ]:
date_range = pd.date_range(start=min_month.to_timestamp(), end=max_month.to_timestamp(), freq='ME')
all_months = pd.DataFrame({'date': date_range})
all_months["date"] = all_months["date"].dt.to_period('M')
all_months

In [ ]:
# Unique cik values (excluding NaN)
ciks = analyst_ratings['cik'].dropna().unique()

# Create all combinations of cik and all months
full_index = pd.MultiIndex.from_product([ciks, all_months['date']], names=['cik', 'date']).to_frame(index=False)

# Merge with original data
test = full_index.merge(analyst_ratings, on=['cik', 'date'], how='left')

# Sort and forward-fill
test = test.sort_values(['cik', 'date'])
test[['mean_rating', 'rating', 'security']] = test.groupby('cik')[['mean_rating', 'rating', 'security']].ffill()


test.reset_index(drop=True, inplace=True)
test